### installs

In [ ]:
%pip install --upgrade pip
%pip install langchain-community langchain-ollama langchain-openai PyMuPDF
%pip install --upgrade --quiet  azure-search-documents
%pip install --upgrade --quiet  azure-identity


### Imports


In [6]:
import logging
from azure.search.documents import SearchClient
from azure.core.credentials import AzureKeyCredential
from langchain_openai import AzureChatOpenAI


# Configuração do logger
logging.basicConfig(level=logging.INFO,
                    format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)


In [ ]:
logging.getLogger("azure").setLevel(logging.WARNING)
logging.getLogger("langchain").setLevel(logging.WARNING)


### envs

In [16]:

AZURE_SEARCH_API_URL = ""
AZURE_SEARCH_API_KEY = ""
AZURE_SEARCH_API_INDEX_NAME= ""

OPENAI_API_KEY =  ""
OPENAI_API_URL =  ""
OPENAI_API_VERSION =  "" 
OPENAI_API_MODEL =  "" 


### Instantiations

In [18]:
llm = AzureChatOpenAI(
    api_key=OPENAI_API_KEY,
    azure_endpoint=OPENAI_API_URL,
    deployment_name=OPENAI_API_MODEL,
    api_version=OPENAI_API_VERSION,
    verbose=False
)

### az search

In [ ]:

search_client = SearchClient(
    endpoint=AZURE_SEARCH_API_URL,
    index_name=AZURE_SEARCH_API_INDEX_NAME,
    credential=AzureKeyCredential(AZURE_SEARCH_API_KEY)
)

query = """
python ai expirience in the document.
   
"""

az_search_results = search_client.search(query)
for result in az_search_results:
    print(result)



### LLM

In [ ]:
# Concatenates the page content of all search results into a single string.
az_search_response = " ".join([result.page_content for result in az_search_results])

messages = [
    (
    "system",
    f"""
        You are an AI specialized in analysis.

        Questions: {query}.
        Context: {az_search_response}.
    """
    ),
    ("human", query), 
]

ai_msg = llm.invoke(messages)
print("AI: ", ai_msg.content)